In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.figure_factory as ff
import json
import geojson
import requests
import plotly.graph_objs as go
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%matplotlib notebook
pd.options.display.max_columns = 150
pd.options.display.max_rows = 2000
import os
os.chdir("/Users/charlessimpson/Library/Mobile Documents/com~apple~CloudDocs/Documents/WorkspacePython/DATA/SQL/")


# Insertion du dataset Pricing

In [18]:
df2000=pd.read_csv("deces-2000.csv", sep=';', encoding='utf-8')
df2001=pd.read_csv("deces-2001.csv", sep=';', encoding='utf-8')
df2002=pd.read_csv("deces-2002.csv", sep=';', encoding='utf-8')
df2003=pd.read_csv("deces-2003.csv", sep=';', encoding='utf-8')
df2004=pd.read_csv("deces-2004.csv", sep=';', encoding='utf-8')
df2005=pd.read_csv("deces-2005.csv", sep=';', encoding='utf-8')
df2006=pd.read_csv("deces-2006.csv", sep=';', encoding='utf-8')
df2007=pd.read_csv("deces-2007.csv", sep=';', encoding='utf-8')
df2008=pd.read_csv("deces-2008.csv", sep=';', encoding='utf-8')
df2009=pd.read_csv("deces-2009.csv", sep=';', encoding='utf-8')
df2010=pd.read_csv("deces-2010.csv", sep=';', encoding='utf-8')
df2011=pd.read_csv("deces-2011.csv", sep=';', encoding='utf-8')
df2012=pd.read_csv("deces-2012.csv", sep=';', encoding='utf-8')
df2013=pd.read_csv("deces-2013.csv", sep=';', encoding='utf-8')
df2014=pd.read_csv("deces-2014.csv", sep=';', encoding='utf-8')
df2015=pd.read_csv("deces-2015.csv", sep=';', encoding='utf-8')
df2016=pd.read_csv("deces-2016.csv", sep=';', encoding='utf-8')
df2017=pd.read_csv("deces-2017.csv", sep=';', encoding='utf-8')
df2018=pd.read_csv("deces-2018.csv", sep=';', encoding='utf-8')
df2019=pd.read_csv("deces-2019.csv", sep=';', encoding='utf-8')
df2020=pd.read_csv('deces_2020.csv', sep=';', encoding='utf-8')
df_01_2021 = pd.read_csv('Deces_2021_M01.csv', sep=';', encoding='utf-8')
df_02_2021 = pd.read_csv('Deces_2021_M02.csv', sep=';', encoding='utf-8')
df_03_2021 = pd.read_csv('Deces_2021_M03.csv', sep=';', encoding='utf-8')
df_04_2021 = pd.read_csv('Deces_2021_M04.csv', sep=';', encoding='utf-8')
df_05_2021 = pd.read_csv('Deces_2021_M05.csv', sep=';', encoding='utf-8')
df_06_2021 = pd.read_csv('Deces_2021_M06.csv', sep=';', encoding='utf-8')
df_07_2021 = pd.read_csv('Deces_2021_M07.csv', sep=';', encoding='utf-8')
df_08_2021 = pd.read_csv('Deces_2021_M08.csv', sep=';', encoding='utf-8')
df_09_2021 = pd.read_csv('Deces_2021_M09.csv', sep=';', encoding='utf-8')
df_10_2021 = pd.read_csv('Deces_2021_M10.csv', sep=';', encoding='utf-8')
df_11_2021 = pd.read_csv('Deces_2021_M11.csv', sep=';', encoding='utf-8')
df_12_2021 = pd.read_csv('Deces_2020_M12.csv', sep=';', encoding='utf-8')

# Je concat toutes les BDD
data = pd.concat([df2000,df2001,df2002,df2003,df2004,df2005,df2006,df2007,df2008,df2009,df2010,df2011,df2012,df2013,df2014,df2015,df2016,df2017,df2018,df2019,df2020,df_01_2021,df_02_2021,df_03_2021,df_04_2021,df_05_2021,
                  df_06_2021,df_07_2021,df_08_2021,df_09_2021,df_10_2021,df_11_2021,df_12_2021])

# je récupère le numéro du département à 2 chiffres (attention aux 10 premiers départements)
def get_department(postal_code):
    postal_code = str(postal_code)
    return postal_code[:2]

# Renomage des colonnes
db = data.rename(columns={'nomprenom':'nom',
                        'datenaiss':'date_naiss',
                        'lieunaiss':'dpt_lieu_naiss',
                        'sexe':'sexe',
                        'commnaiss':'ville',
                        'paysnaiss':'pays',
                        'datedeces':'date_dc',
                        'lieudeces':'dpt_lieu_dc',
                        'actedeces':'dpt_lieu_deces'})

# je réduis ma BDD à la taille utile
#db = db[['nom','dpt_lieu_dc', 'date_naiss','date_dc']]

# je récupère les numéros de départements simplifiés pour les décès
db['dpt_deces'] = db['dpt_lieu_dc'].apply(lambda x : get_department(x))
print("Nombre de deces cumulés sur la période",db.shape)

# Nettoyage des numéros de départements pour s'assurer qu'on récupère correctement les 10 premiers départements
def non_int_filter(dpt_deces):
    try:
        return int(dpt_deces)
    except:
        return 0
db['dpt_deces'] = db['dpt_deces'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_deces']== 0]


# je récupère les numéros de départements simplifiés pour les naissance
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : get_department(x))
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_lieu_naiss']== 0]


# J'identifie les décès sans département
db = db.loc[db['dpt_deces']!= 0] # fonction qui supprimer les lignes sans départements
# transformer la data décès au bon format pour manipuler cette donnée 
db['date_deces']=pd.to_datetime((db.date_dc), format='%Y%m%d', errors='coerce')

# Nettoyage du format de la date de décès, pour avoir le mois et l'année au bon format
db=db.dropna(subset=['date_deces'])

def year(date_dc):
    date_dc = str(date_dc)
    return date_dc[:4]
db['year'] = db['date_dc'].apply(lambda x : year(x))

def month(date_dc):
    date_dc = str(date_dc)
    return date_dc[:6]
db['month'] = db['date_dc'].apply(lambda x : month(x))

#Suppression des DOMTOM
db=db[db['dpt_deces']!=97]
db=db[db['dpt_deces']!=98]
db=db[db['dpt_deces']!=99]

# Je rajoute la BDD des départements pour insérer les noms des départements
departements_info = pd.read_excel('departements-francais.xls')

number_to_departement = departements_info.set_index('NUMÉRO')['NOM'].to_dict()

# Nettoyage des déparmtents, j'enlève l'outre mer et je rajouter le nom des départements
number_to_departement[97] =  'Outre-mer'
db = db.rename(columns={'dpt_deces':'departement_number'})
db['departement'] = db['departement_number'].apply(lambda x : number_to_departement.get(x, ""))
wrong_dpt = db[db.departement_number.isin([98, 99])]
db = db[~db.departement_number.isin([98, 99])]

# je modifie les noms des départements qui ne sont pas correctes
db['departement'] = db['departement'].str.replace("Val-D'Oise", "Val-d'Oise")
db['departement'] = db['departement'].str.replace( 'Seine-St-Denis', "Seine-Saint-Denis")
db['departement'] = db['departement'].str.replace( "Côtes d'Armor", "Côtes-d'Armor")
db['departement'] = db['departement'].str.replace( "Vandée", "Vendée")


db.sample(3)


Nombre de deces cumulés sur la période (12788539, 10)


,nom,sexe,date_naiss,dpt_lieu_naiss,ville,pays,date_dc,dpt_lieu_dc,dpt_lieu_deces,departement_number,date_deces,year,month,departement
101272,RAMBOURG*CHARLES ELIE LEON/,1,19231022,8,BALHAM,NaN,20010220.0,51454,464,51,2001-02-20,2001,200102,Marne
59183,MARICOURT*ANDREA JEANNINE LUCIE/,2,19250329,18,SANCOINS,NaN,20120201.0,21584,15,21,2012-02-01,2012,201202,Côte-d'Or
501833,FANG*JOSETTE PIERRETTE/,2,19270912,75,BOULOGNE-BILLANCOURT,NaN,20161027.0,17461,447,17,2016-10-27,2016,201610,Charente-Maritime


In [22]:
# création du DATA qui permet de générer les différents graphiques. nous nous intéressons à 2000, 2010, 2020 et 2021. Att il y a des chiffres entre ces années qui se remontés par le skype 
# ne pas y faire attention


db=db[db['year']>'1999']
db_year=db.pivot_table(index='departement',columns='year', values='nom', aggfunc='count').fillna(0).reset_index()
#db_year.loc['total']=db_year.sum(axis=0)
db_year['var_2021_2020'] = round((db_year['2020'] - db_year['2020']) / db_year['2020']* 100,1)
db_year['var_2021_2010'] = round((db_year['2020'] - db_year['2010']) / db_year['2020']* 100,1)
db_year['var_2021_2000'] = round((db_year['2020'] - db_year['2000']) / db_year['2020']* 100,1)
db_year=db_year[['departement','2000','2010','2020','2021','var_2021_2010']]
db_year.tail(5)

year,departement,2000,2010,2020,2021,var_2021_2010
90,Vendée,5213.0,5757.0,7704.0,6505.0,25.3
91,Vienne,4245.0,4407.0,5549.0,4472.0,20.6
92,Vosges,3841.0,3746.0,5370.0,3895.0,30.2
93,Yonne,3925.0,3798.0,4876.0,3785.0,22.1
94,Yvelines,8054.0,7960.0,10895.0,7820.0,26.9


# DATA décès par année

In [23]:
# Fichier à télécharger ici : https://github.com/gregoiredavid/france-geojson/blob/master/departements.geojson

with open ("departements.geojson", "r") as f:
    departements = geojson.load(f)
    
with open ("villes_principales_france.geojson", "r") as f2:
    villes_principales = geojson.load(f2)
    
villes_principales = villes_principales.features

with open ("idf.geojson", "r") as f3:
    idf = geojson.load(f3)

for i in range(1, len(db_year.columns)):
    print(db_year.columns[i])

def generateMaps(columns, fileName, description, title, geojson):
    # on génère les cartes en fonction des colonnes présentes
    for i in range(1, len(columns)):
        
        column = columns[i]
        dataSet = db_year

        # on récupère les départements non présents
        deps=[]
        for i, d in dataSet.iterrows():
            deps.append(d['departement'])
        for dpt in departements.features:
            if dpt["properties"]["nom"] not in deps:
                dataSet = dataSet.append({"departement": dpt["properties"]["nom"], column: 0}, ignore_index = True)
        
        # si la carte qu'on cherche à générer est celle de l'IDF, on modifie le dataFrame
        if 'idf' in fileName:
            baseDataSet = dataSet
            dataSet = pd.DataFrame([], columns=['departement', column])
            for i, d in baseDataSet.iterrows():
                x = d['departement']
                if x == 'Paris' or x == 'Val-de-Marne' or x == 'Val-d\'Oise' or x == 'Seine-Saint-Denis' or x == 'Essonne' or x == 'Hauts-de-Seine' or x == 'Yvelines' or x == 'Seine-et-Marne':
                    dataSet.loc[i] = [x, d[column]]
        
        prefix = ''
        # si la colonne contient un agrégat comme "50y", on ajoute " sur 50 ans" au titre de la carte (fonctionne avec n'importe quel nombre)
        # sinon, on laisse vide pour ne pas ajouter un suffixe au titre des autres cartes
        columnDesc = ''
        if column[-1] == 'y' and column[len(column) - 2].isdigit() and column[len(column) - 3].isdigit():
            columnDesc = ' sur ' + column[0] + column[1] + ' ans'
            prefix = column + '-'
        
        # on définit la fig de la carte à générer
        fig = px.choropleth(
            dataSet,
            geojson=geojson,
            locations='departement',
            scope='europe',
            color=dataSet[column],
            range_color=(dataSet[column].min(), dataSet[column].max()),
            featureidkey="properties.nom",
            hover_data=[column],
            # change
            labels={column: description + columnDesc, 'departement': 'Département'}
        )

        # on affiche les villes principales sur la fig avec le geojson chargé précédemment
        for ville in villes_principales:
            # if carte IDF paris 
            if 'idf' in fileName:
                if ville.properties['nom'] == 'Paris':
                    fig.add_trace(go.Scattergeo(
                    lon = [ville.geometry.coordinates[0]],
                    lat = [ville.geometry.coordinates[1]],
                    text = ville.properties['nom'],
                    textfont=dict(
                        size=12,
                        color="white"
                    ),
                    mode="text",
                    showlegend = False,
                    hoverinfo="skip"
                ))
            else:
                fig.add_trace(go.Scattergeo(
                    lon = [ville.geometry.coordinates[0]],
                    lat = [ville.geometry.coordinates[1]],
                    text = ville.properties['nom'],
                    textfont=dict(
                        size=12,
                        color="white"
                    ),
                    mode="text",
                    showlegend = False,
                    hoverinfo="skip"
                ))

        fig.update_layout(title_text = title + columnDesc)
        fig.update_layout(margin=dict(l=40, r=40, t=40, b=40))       
        fig.update_geos(fitbounds="locations", visible=False)

        # génération du fichier de la carte donnée
        fig.write_html("cartes-inter/" + prefix + fileName + ".html")
        fig.show()

generateMaps(db_year.columns, "concessions-departements", "Prix moyen", "Prix moyen des concessions dans les départements de France", departements)
# le fichier cartes-inter/concessions-departements-yc-idf.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

generateMaps(db_year.columns, "concessions-idf", "Prix moyen", "Prix moyen des concessions dans les départements d'Ile-de-France", idf)

2000
2010
2020
2021
var_2021_2010


FileNotFoundError: [Errno 2] No such file or directory: 'cartes-inter/concessions-departements.html'

# Analyse des décès par âge moyen

In [ ]:
# je calcu l'age de décès du défunt
db['diff_in_D'] = abs(db['date_dc'] - db['date_naiss'])
db['diff_in_Y']=db['diff_in_D']/10000
db.sample(3)

In [ ]:
# evolution du nombre de décès et l'âge moyen
db_age_moyen = db.groupby(['year'])['diff_in_Y'].agg(['count','mean']).reset_index()
db_age_moyen.tail(2)

In [ ]:
import plotly  
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=db_age_moyen["year"], y=db_age_moyen["count"], name="Evolution du nombre de décès"),
    secondary_y=False, 
)

fig.add_trace(
    go.Scatter(x=db_age_moyen["year"], y=db_age_moyen["mean"], name="Evolution de l'âge moyen"),
    secondary_y=True, 
)

# Add figure title
fig.update_layout(
    title_text="Nombre de décès par mois",
)

'''# Set x-axis title
fig.update_xaxes(title_text="% évolution décès par rapport à l'année précédente")'''

'''# Set y-axes titles
fig.update_yaxes(title_text="<b>Nombre décès par mois</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>% par rapport à l'année précédente</b> ", secondary_y=True)'''

fig.write_html("cartes-inter/pie_chart_evol_deces.html")


fig.show()

In [ ]:
# age moyen par département et par année
db_age_moyen = db.pivot_table(index='departement', columns='year', values='diff_in_Y', aggfunc='mean').fillna(0)
db_age_moyen=pd.DataFrame(db_age_moyen).reset_index()
db_age_moyen.sample(2)

# Calcul nombre personnes décédées dans le meme département


In [ ]:
db.sample(2)

In [ ]:
comparison_column = np.where(db["dpt_lieu_naiss"] == db["departement_number"], True, False)
print(comparison_column)
db["equal"] = comparison_column

db.sample(10)

In [ ]:
# calcul du pourcentage des personnes nées dans le département 
db_ptc=db.pivot_table(index='equal', columns='year', values='nom', aggfunc='count').reset_index()
db_ptc = db_ptc.div(db_ptc.sum(axis=0), axis=1)
db_ptc

In [ ]:
# calcul des pourcentages des personnées dans le meme département
db_2020=db[db['year']=='2020']
#db_2020_dpt_ptc = db_2020.groupby('departement')['equal'].count()#.fillna(0)
db_2020_dpt_ptc = db_2020.pivot_table(index='departement', columns='equal', values='nom', aggfunc='count').reset_index()

db_2020_dpt_ptc['Total'] = db_2020_dpt_ptc[True]+db_2020_dpt_ptc[False] 
db_2020_dpt_ptc['Ptc_total'] = round((db_2020_dpt_ptc[True] / db_2020_dpt_ptc['Total'])*100,1)
db_2020_dpt_ptc#.sample(4)

In [ ]:
db_year.sample(2)

In [ ]:
# Fichier à télécharger ici : https://github.com/gregoiredavid/france-geojson/blob/master/departements.geojson
# L'idée est de pouvoir piocher différents colonnes du dataset ci-dessous 


with open ("geojson/departements.geojson", "r") as f:
    departements = geojson.load(f)

with open ("geojson/villes_principales_france.geojson", "r") as f2:
    villes_principales = geojson.load(f2)
villes_principales = villes_principales.features

# pour pouvoir générer déjà les graphique, j'ai créé une variabl
year='Ptc_total'
# on définit la fonction qui génère la carte selon le dataset donné

# idéalement sur chaque graphique on peut rajouter une ou 2 autres valeurs à montrer sur le hover (valeur qu'on voit lorsqu'on regarde un département)
# rajouter une phrase + valeur qui est la somme de la colonne (ainsi je peux dire: le total des décès en 2021 ou autre valeur ) est de X décès
# attention le titre du graphique doit pouvoir être personnalisé
def generateDepartementsMap(dataSet, fileName, years):
    attribute = year
    
    # on récupère les départements non présents
    deps=[]
    for i, d in dataSet.iterrows():
        deps.append(d['departement'])
    for dpt in departements.features:
        if dpt["properties"]["nom"] not in deps:
            dataSet = dataSet.append({"departement": dpt["properties"]["nom"], year: 0}, ignore_index = True)
            print('département manquant:', dpt["properties"]["nom"])
            
    # on définit la fig de la carte à générer
    fig = px.choropleth(
        dataSet,
        geojson=departements,
        locations='departement',
        scope='europe',
        color=dataSet[attribute],
        range_color=(dataSet[attribute].min(), dataSet[attribute].max()),
        featureidkey="properties.nom",
        hover_data=[attribute,],
        labels={'price_value':'Nombre de décès', 'departement': 'Département'}
    )
    
    # on affiche les villes principales sur la fig avec le geojson chargé précédemment
    for ville in villes_principales:
        fig.add_trace(go.Scattergeo(
            lon = [ville.geometry.coordinates[0]],
            lat = [ville.geometry.coordinates[1]],
            text = ville.properties['nom'],
            textfont=dict(
                size=12,
                color="white"
            ),
            mode="text",
            showlegend = False,
            hoverinfo="skip"
        ))
        
    fig.update_layout(title_text = "Pourcentage des défunts nés dans le même département")
    fig.update_layout(margin=dict(l=40, r=40, t=40, b=40))       
    fig.update_geos(fitbounds="locations", visible=False)
    
    # génération du fichier de la carte donnée
    fig.write_html(fileName)
    fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
# créer une page pour 2000
#generateDepartementsMap(db_year, "cartes-inter/db_2000_dpt_count-departements.html", "2000")
# créer une page pour 2010
#generateDepartementsMap(db_year, "cartes-inter/db_2010_dpt_count-departements.html", "2010")
# créer une page pour 2020
#generateDepartementsMap(db_year, "cartes-inter/db_2020_dpt_count-departements.html", "2020")
# créer une page pour qui montre les variations entre 2021 et 2020
generateDepartementsMap(db_2020_dpt_ptc, "cartes-inter/db_2020_dpt_ptc-departements.html", "pourcentage des défunts nés dans le meme département")

#generateDepartementsMap(db_year, "cartes-inter/db_2021_dpt_count-departements.html", "2021")

# le fichier cartes-inter/concessions-departements.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus
###### on duplique le code pour IDF####
# on charge le geojson de l'Ile-de-France
'''with open ("geojson/idf.geojson", "r") as f3:
    idf = geojson.load(f3)

# on définit la fonction qui récupère les datasets IDF et génère la carte pour l'agrégat donné
def generateIdfMap(baseDataSet, fileName, years):
    attribute = 'price_value'
    
    # on crée un dataSet en gardant que les départements d'IDF
    newDataSet = pd.DataFrame([], columns=['departement', 'price_value'])
    for i, d in baseDataSet.iterrows():
        x = d['departement']
        if x == 'Paris' or x == 'Val-de-Marne' or x == 'Val-d\'Oise' or x == 'Seine-Saint-Denis' or x == 'Essonne' or x == 'Hauts-de-Seine' or x == 'Yvelines' or x == 'Seine-et-Marne':
            newDataSet.loc[i] = [x, d['price_value']]
            
    # on définit la fig de la carte IDF à générer
    fig = px.choropleth(
        newDataSet,
        geojson=idf,
        locations='departement',
        scope='europe',
        color=newDataSet[attribute],
        range_color=(newDataSet[attribute].min(), newDataSet[attribute].max()),
        featureidkey="properties.nom",
        fitbounds='geojson',
        projection='mercator',
        hover_data=[attribute],
        labels={'Nombre de décès':'Nombre de décès', 'departement': 'Département'}
    )
    
    fig.update_layout(title_text = "")
    fig.update_layout(margin=dict(l=40, r=40, t=40, b=40), showlegend=False, width=400, height=400)
    fig.update_coloraxes(showscale=False)
    fig.update_geos(fitbounds="locations", visible=False)
    
    fig.show()
    fig.write_html(fileName)'''

    # on appelle la fonction qui génère une carte de l'Ile-de-France pour le dataset donné en paramètre
#generateIdfMap(db_year, 'cartes-inter/db_2000_dpt_count-idf.html', '2000')
#generateIdfMap(db_year, 'cartes-inter/db_2010_dpt_count-idf.html', '2010')
#generateIdfMap(db_year, 'cartes-inter/db_2020_dpt_count-idf.html', '2020')
#generateIdfMap(db_year, 'cartes-inter/db_2021_dpt_count-idf.html', '2021')
#generateIdfMap(db_year, "cartes-inter/db_2020_dpt_count-departements.html", "variation 2021 versus 2020")


#var_2021_2020
# le fichier cartes-inter/concessions-idf.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus